## Step 1: Configure Google Cloud Authentication

For Vertex AI models, you need to authenticate with Google Cloud.

In [ ]:
import os

# TODO(developer): Replace with your Google Cloud Project ID
project_id = "your-project-id"  # e.g., "my-dnb-project"

# Set the project for gcloud (if running locally)
!gcloud config set project {project_id}

# Verify authentication
!gcloud auth application-default login

## Step 2: Connect to Toolbox Server

Verify the Toolbox server is running and accessible.

In [4]:
from toolbox_langchain import ToolboxClient
client = ToolboxClient("http://localhost:5000")
tools = await client.aload_toolset("dnb-tools")
print(tools)  # List available tools
# Use a tool
result = await tools["dnb-echo-helloworld"].ainvoke({"helloWorld": "Testing DNB API"})
print(result)  # Print the result of the tool invocation

[ToolboxTool(name='dnb-echo-helloworld', description='Test DNB API connectivity by calling the echo endpoint', args_schema=<class 'toolbox_core.utils.dnb-echo-helloworld'>), ToolboxTool(name='dnb-get-metadata', description='Retrieve metadata about available statistical datasets from DNB', args_schema=<class 'toolbox_core.utils.dnb-get-metadata'>), ToolboxTool(name='dnb-query-statistics', description='Query Dutch banking statistics data with optional filters\n\nArgs:\n    limit (Optional): Maximum number of results to return\n    offset (Optional): Number of results to skip', args_schema=<class 'toolbox_core.utils.dnb-query-statistics'>)]


TypeError: list indices must be integers or slices, not str

In [ ]:
import requests

TOOLBOX_URL = "http://localhost:5000"

# Check server health
try:
    response = requests.get(f"{TOOLBOX_URL}/health")
    print(f"✅ Toolbox server is running: {response.status_code}")
except Exception as e:
    print(f"❌ Cannot connect to Toolbox server: {e}")
    print("Make sure to run 'docker-compose up -d' in C:\\Users\\rjjaf\\_Projects\\orkhon\\backend\\toolbox\\")

In [ ]:
# List available tools
try:
    response = requests.get(f"{TOOLBOX_URL}/api/tools")
    tools = response.json()
    print("Available tools:")
    for tool_name, tool_info in tools.items():
        print(f"  - {tool_name}: {tool_info.get('description', 'No description')}")
except Exception as e:
    print(f"Error loading tools: {e}")

## Step 3: Load the DNB Echo Tool

Load only the `dnb-echo-test` tool from the Toolbox server.

In [ ]:
from toolbox_langchain import ToolboxClient

async def load_echo_tool():
    async with ToolboxClient(TOOLBOX_URL) as client:
        # Load the specific echo tool
        echo_tool = await client.aload_tool("dnb-echo-test")
        return [echo_tool]

# Load the tool
import asyncio
tools = await load_echo_tool()

print(f"Loaded {len(tools)} tool(s):")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")

## Step 4: Test Direct Tool Invocation

Before creating an agent, test calling the tool directly.

In [ ]:
# Test direct invocation
async def test_echo_directly():
    async with ToolboxClient(TOOLBOX_URL) as client:
        echo_tool = await client.aload_tool("dnb-echo-test")
        
        # Invoke the tool (Echo API doesn't require parameters)
        result = await echo_tool.ainvoke({})
        
        print("Direct invocation result:")
        print(result)

await test_echo_directly()

## Step 5: Create a Simple Agent

Create a LangGraph agent that can use the Echo tool.

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_google_vertexai import ChatVertexAI
from langgraph.checkpoint.memory import MemorySaver

# System prompt for the agent
system_prompt = """
You are a helpful assistant that can test DNB API connectivity.
You have access to the DNB Echo API tool which returns "Hello World!" 
to verify the connection is working.

When asked to test the connection, use the dnb-echo-test tool.
"""

async def create_simple_agent():
    # Initialize the LLM
    model = ChatVertexAI(
        model_name="gemini-2.0-flash-001",
        project=project_id,
        location="us-central1"
    )
    
    # Load the echo tool
    async with ToolboxClient(TOOLBOX_URL) as client:
        tools = [await client.aload_tool("dnb-echo-test")]
    
    # Create the agent
    agent = create_react_agent(
        model,
        tools,
        checkpointer=MemorySaver()
    )
    
    return agent

agent = await create_simple_agent()
print("✅ Agent created successfully!")

## Step 6: Test the Agent with Queries

Run test queries to verify the agent can use the Echo tool.

In [ ]:
test_queries = [
    "Can you test the DNB API connection?",
    "Use the echo tool to verify connectivity",
    "Is the DNB API responding?"
]

config = {"configurable": {"thread_id": "test-thread-1"}}

async def run_agent_tests():
    for query in test_queries:
        print(f"\n{'='*60}")
        print(f"Query: {query}")
        print('='*60)
        
        inputs = {"messages": [("user", system_prompt + "\n\n" + query)]}
        response = await agent.ainvoke(inputs, config=config)
        
        print(f"Agent Response: {response['messages'][-1].content}")
        print()

await run_agent_tests()

## Step 7: Interactive Testing

Use this cell to test custom queries interactively.

In [ ]:
async def ask_agent(question: str):
    """
    Ask the agent a question interactively.
    
    Args:
        question: Your question to the agent
    """
    inputs = {"messages": [("user", system_prompt + "\n\n" + question)]}
    response = await agent.ainvoke(inputs, config=config)
    return response['messages'][-1].content

# Example usage:
# result = await ask_agent("Test the DNB Echo API for me")
# print(result)

## Next Steps

Once the Echo API is working, you can:

1. **Add more tools** by updating `config/tools.dev.yaml`
2. **Test Statistics API** - Query DNB banking statistics
3. **Test Public Register API** - Search for licensed institutions
4. **Add error handling** - Handle API rate limits and failures
5. **Add observability** - View traces in Jaeger at http://localhost:16686

See the main README at `C:\Users\rjjaf\_Projects\orkhon\backend\toolbox\README.md` for more details.

## Troubleshooting

**Connection refused errors:**
```bash
# Check if Toolbox is running
docker ps | findstr genai-toolbox-mcp

# View logs
docker-compose logs genai-toolbox-mcp

# Restart the container
docker-compose restart genai-toolbox-mcp
```

**Authentication errors (401):**
- Verify `DNB_SUBSCRIPTION_KEY_DEV` in `.env` file
- Check key at https://api.portal.dnb.nl/ → Profile → Subscriptions
- Ensure `DNB_ENVIRONMENT=dev` matches the key being used